# TDE-v2 Dataset Cookbook

This cookbooks severs as the preparation for the TDE-v2 dataset for the Chat-Transform project. It refactored the original dataset from Yeye-He's [repo](https://github.com/Yeye-He/Transform-Data-by-Example/tree/master/Benchmark) into a fine-tuning `Alpaca` format that is compatible with the [LLaMA-Factory](https://github.com/hiyouga/LLaMA-Factory)


In [ ]:
import os
import pandas as pd
import json

## Load and Check

In [ ]:
data_directory = '../data/TDE-v2/'
data_frames = []

# Load all JSON files in the directory and convert to DataFrame
for subdir, _, files in os.walk(data_directory):
    for file in files:
        if file.endswith('.json'):
            file_path = os.path.join(subdir, file)
            with open(file_path, 'r') as f:
                df = pd.json_normalize(json.load(f),
                                       meta=[['context', 'input'], ['context', 'output']])
                
                test_fp = file_path.replace(data_directory, "")
                df['test_path'] = os.path.splitext(test_fp)[0]  # remove file extension
                data_frames.append(df)

# Combine all DataFrames into a single DataFrame
df_all = pd.concat(data_frames, ignore_index=True)

# split the function from field 'instruction'
df_all['function'] = df_all['instruction'].apply(lambda x: x.split(':')[0].strip())

# rename context.input and context.output to input and output
df_all.rename(columns={'context.input': 'ctx_input', 'context.output': 'ctx_output'}, inplace=True)

# check the shape of the dataframe
df_all.shape

(226, 7)

In [ ]:
df_all.head()

,chat,instruction,tuples,ctx_input,ctx_output,test_name,function
0,Transform first and last names into username,format(): Combine first letter of first name w...,"[{'input': 'john smith', 'output': 'jsmith'}, ...",First and last names separated by a tab charac...,Username created by combining the first letter...,benchmark-FF-Trifacta-GoogleRefine/example_fil...,format()
1,Extract name of actor/actress,extract(): extract name of actor/actress from ...,[{'input': '* '''1953 [[Meena Kumari]] – ''[[B...,text format containing movie details,extracted name of actor/actress,benchmark-FF-Trifacta-GoogleRefine/example_fil...,extract()
2,Extract character names from wiki-style list,extract(): Extract the character name from the...,[{'input': '* '''1953 [[Meena Kumari]] – ''[[B...,Wiki-style list item containing actor and char...,Character name extracted from the input,benchmark-FF-Trifacta-GoogleRefine/example_fil...,extract()
3,format date of birth,format(): convert YYYYMMDD to MM-DD-YYYY,"[{'input': '19610223', 'output': '02-23-1961'}...",date in YYYYMMDD format,formatted date in MM-DD-YYYY,benchmark-FF-Trifacta-GoogleRefine/example_fil...,format()
4,Extract and format movie titles,transform(): Extract the movie title from the ...,[{'input': '* '''1953 [[Meena Kumari]] – ''[[B...,String containing movie information with title...,Lowercase movie title,benchmark-FF-Trifacta-GoogleRefine/example_fil...,transform()


> TDE-v2 dataset Data Dictionary

- The `chat` field represents a conversation between a user and an AI assistant. It is expected to be the input to the fine-tined LM.

- The `instruction` field represents a coding instruction with API-style `function` name that specifies the transformation type. It is expected to be the target output from the fine-tined LM.

- The `tuples` field represents a list of transformation-pairs, which serves as additional few-shot examples to help the fine-tined LM to understand the user's intention. Each transformation-pair is a list of tuples, where each tuple contains an input and an output. The context is stored in `ctx_input` and `ctx_output` fields, respectively.

- `test_path` refers to the original path of the test case in the TDE dataset.



In [4]:
# check duplicates of instruction, and sort it
df_all[df_all.duplicated(subset=['instruction'], keep=False)].sort_values(by='instruction')


,chat,instruction,tuples,ctx_input,ctx_output,test_name,function


In [5]:
# show distribution of function
df_all['function'].value_counts()


extract()             57
unit_convert()        48
format()              45
domain_calculate()    34
transform()           25
domain_map()          17
Name: function, dtype: int64

In [6]:
# count the number of transformation-pairs in tuples
df_all['num_tuples'] = df_all['tuples'].apply(len)
print(df_all['num_tuples'].describe()) 

print("Total number of tuples: ", df_all['num_tuples'].sum())
# show 

count    226.000000
mean       5.575221
std        2.232309
min        4.000000
25%        5.000000
50%        5.000000
75%        5.000000
max       25.000000
Name: num_tuples, dtype: float64
Total number of tuples:  1260


In [7]:

# re arrange columns
df_all = df_all[['test_path', 'chat', 'instruction', 'ctx_input', 'ctx_output', 'tuples']]

# sort by test_path
df_all.sort_values(by='test_path', inplace=True)


In [8]:
# export to CSV
df_all.to_csv('../data/TDE-v2/tde-v2-all.csv', index=False)

## Refactor to Alpaca Format